<a href="https://colab.research.google.com/github/pic980/Ai-Agent/blob/main/Copy_of_FINAL_VTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers hugchat transformers accelerate safetensors gradio -q
!pip install git+https://github.com/huggingface/diffusers -q
!pip install ipywidgets invisible_watermark -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
from diffusers import StableDiffusionXLPipeline
import torch

# Load Stable Diffusion XL pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    "segmind/SSD-1B",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)
pipe.to("cuda")

# Load Whisper speech-to-text model
whisper = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
    chunk_length_s=30,
    device="cuda:0"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

text_encoder_2/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import gradio as gr

# Function to handle transcription and image generation
def generate_image(audio, style, quality, render, angle, lighting, background, device, emotion):
    # Save audio input and transcribe
    transcription = ""
    if audio:  # Ensure audio input exists
        transcription = whisper(audio)['text']

    # Build the prompt
    prompt = (
        f"A stunning {style}, {quality} shot of {transcription} captured in {device} using {angle} "
        f"and rendered by {render}, illuminated by {lighting} light, with {emotion} emotions in a {background} background setting."
    )
    neg_prompt = "ugly, blurry, poor quality, deformed structure, very bad lighting, bad colouring, noise"

    # Generate the image
    generated_image = pipe(prompt=prompt, negative_prompt=neg_prompt).images[0]
    generated_image.save("generated_image.png")

    return transcription, "generated_image.png"

# Gradio interface
interface = gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Audio(type="filepath", label="Record or Upload Audio"),  # Corrected type
        gr.Dropdown(
            ["photorealistic", "low poly", "cinematic", "cartoon", "pixel art", "graffiti", "sketching"],
            label="Style"
        ),
        gr.Dropdown(
            ["High resolution", "8K", "clear", "pixelated NFT", "heavy detailed", "beautiful", "realistic+++", "hyper detailed", "masterpiece"],
            label="Quality"
        ),
        gr.Dropdown(
            ["Pixar", "Octane", "real-time ray tracing", "Christopher Nolan", "James Cameron", "unreal engine", "unity"],
            label="Render by"
        ),
        gr.Dropdown(
            ["Wide-angle lens", "full shot", "Top angle", "Telephoto lens", "Prime lens", "Zoom lens", "Macro lens", "Fisheye lens", "Tilt-shift lens", "Portrait lens", "Anamorphic lens", "Cinematic lens", "Fixed focal length lens", "Variable focal length lens"],
            label="Angle"
        ),
        gr.Dropdown(
            ["Soft", "ambient", "ring light", "neon", "Natural", "Soft", "Harsh", "Dramatic", "Backlit", "Studio"],
            label="Lighting"
        ),
        gr.Dropdown(
            ["outdoor", "indoor", "space", "nature", "sci-fi", "neon", "abstract"],
            label="Background"
        ),
        gr.Dropdown(
            ["Go Pro", "iPhone 15", "Canon EOS R5", "Nikon Z7", "Sony F950", "Drone", "CCTV"],
            label="Device"
        ),
        gr.Dropdown(
            ["Happy", "Sad", "Mysterious", "Surprised", "Annoyed", "Neutral", "Dreamy", "Nostalgic"],
            label="Emotion"
        ),
    ],
    outputs=[
        gr.Textbox(label="Transcribed Text"),
        gr.Image(label="Generated Image"),
    ],
    title="Stable Diffusion XL with Speech-to-Image",
    description="Record your voice, and let the model generate an image based on your description and selected styles!"
)


In [ ]:
# Launch the interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://75b3a28b09dccc8486.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
